In [ ]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark import SparkContext

sc = SparkContext.getOrCreate()

sqlContext = SQLContext(sc)


df_data_1 = sqlContext.read.format('com.databricks.spark.csv').\
options(header='true', inferschema='true').load('2015-summary.csv')

df_data_1.printSchema()
df_data_1.take(5)





In [ ]:
us_from_flights=df_data_1.filter("ORIGIN_COUNTRY_NAME =='United States'")




type(us_from_flights)



In [ ]:
us_from_flights=df_data_1[df_data_1.ORIGIN_COUNTRY_NAME 
                          =='United States']
us_from_flights.show()





In [ ]:
from pyspark.sql.functions import max 
df_data_1.select(max("count")).take(1) 








In [ ]:
#from pyspark.sql.functions import first
us_from_flights=df_data_1.filter("ORIGIN_COUNTRY_NAME =='United States'").\
select("DEST_COUNTRY_NAME", "COUNT")




us_from_flights.show()


In [ ]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import desc

df_flights2015=df_data_1.withColumn("COUNT", df_data_1["COUNT"].cast("double"))
df_flights2015.printSchema()

df_flights2015.groupBy("DEST_COUNTRY_NAME").sum("count").\
withColumnRenamed("sum(count)", "destination_total").\
sort(desc("destination_total")).limit(5).collect() 





In [ ]:
df_data_2 = sqlContext.read.format('com.databricks.spark.csv').\
options(header='true', inferschema='true').load('2014-summary.csv')

df_data_2.take(5)




In [ ]:
df_flights2014=df_data_2.withColumn("COUNT", df_data_2["COUNT"].cast("double")).\
withColumnRenamed("COUNT","COUNT2014")

df_flights2014.take(5)



In [ ]:
df_join=df_flights2014.join(df_flights2015,\
(df_flights2014.DEST_COUNTRY_NAME==df_flights2015.DEST_COUNTRY_NAME)\
&(df_flights2014.ORIGIN_COUNTRY_NAME==df_flights2015.ORIGIN_COUNTRY_NAME)).\
select(df_flights2014.DEST_COUNTRY_NAME,df_flights2014.ORIGIN_COUNTRY_NAME,"COUNT2014","COUNT")



df_join.take(5)



In [ ]:
df_join.head()



In [ ]:
from pyspark.sql.functions import udf
compute_diff=udf(lambda x,y: y-x,DoubleType())



df_result=df_join.select(df_join.DEST_COUNTRY_NAME,\
                         df_join.ORIGIN_COUNTRY_NAME,\
                         compute_diff(df_join.COUNT2014, df_join.COUNT)).\
                         withColumnRenamed("<lambda>(COUNT2014, COUNT)","diff")

df_result.take(5)



In [ ]:
df_result=df_join.select(df_join.DEST_COUNTRY_NAME,\
                         df_join.ORIGIN_COUNTRY_NAME,\
                         df_join.COUNT-df_join.COUNT2014)
df_result.take(5)


In [ ]:
df_outgoingflights=df_flights2015.groupBy("ORIGIN_COUNTRY_NAME").\
sum("COUNT").withColumnRenamed("sum(COUNT)","deptFlights").limit(5)



df_outgoingflights.take(5)